# Imviz compass visualization using Matplotlib

This concept notebook visualizes the compass algorithm that Imviz will use. For simplicity, visualization is done using Matplotlib.

Some of these examples are also in the unit tests for `wcs_utils.py`.

In [ ]:
import gwcs
import numpy as np
from astropy import units as u
from astropy.coordinates import ICRS
from astropy.io import fits
from astropy.modeling import models
from astropy.utils.data import download_file
from astropy.wcs import WCS
from glue.core import Component, Data
from gwcs import coordinate_frames as cf

from jdaviz import Imviz
from jdaviz.configs.imviz import wcs_utils

In [ ]:
%matplotlib inline

## Case 1: Dummy WCS with 10x10 image

In [ ]:
a = np.arange(100).reshape((10, 10))

In [ ]:
w = WCS()

In [ ]:
_ = wcs_utils.draw_compass_mpl(a, wcs=w, show=True)

Now, let's try that in the Compass plugin. We will display this same image in Imviz. Open the Compass plugin to see the result.

In [ ]:
hdu = fits.ImageHDU(a, name='SCI')
hdu.header.update(w.to_header())

In [ ]:
imviz = Imviz()
imviz.load_data(hdu, data_label='arr_with_dummy_wcs')
imviz.show()

## Case 2: Real WCS with no rotation nor distortion

We reuse the array from above but WCS from https://learn.astropy.org/tutorials/celestial_coords1.html example.

In [ ]:
w = WCS({'CTYPE1': 'RA---TAN',
         'CUNIT1': 'deg',
         'CDELT1': -0.0002777777778,
         'CRPIX1': 1,
         'CRVAL1': 337.5202808,
         'NAXIS1': 10,
         'CTYPE2': 'DEC--TAN',
         'CUNIT2': 'deg',
         'CDELT2': 0.0002777777778,
         'CRPIX2': 1,
         'CRVAL2': -20.833333059999998,
         'NAXIS2': 10})

In [ ]:
_ = wcs_utils.draw_compass_mpl(a, wcs=w)

Now, let's try that in the Compass plugin. We will display this same image in Imviz. Open the Compass plugin to see the result.

In [ ]:
hdu = fits.ImageHDU(a, name='SCI')
hdu.header.update(w.to_header())

In [ ]:
imviz = Imviz()
imviz.load_data(hdu, data_label='arr_with_simple_wcs')
imviz.show()

## Case 3: HST/ACS FLT with distortion

We replaced the actual ACS image (`jb5g05ubq_flt.fits`) with random data to avoid data download, but the WCS is real.

In [ ]:
a = np.random.random((2048, 4096))
w = WCS({'WCSAXES': 2,
         'CRPIX1': 2100.0,
         'CRPIX2': 1024.0,
         'PC1_1': -1.14852e-05,
         'PC1_2': 7.01477e-06,
         'PC2_1': 7.75765e-06,
         'PC2_2': 1.20927e-05,
         'CDELT1': 1.0,
         'CDELT2': 1.0,
         'CUNIT1': 'deg',
         'CUNIT2': 'deg',
         'CTYPE1': 'RA---TAN',
         'CTYPE2': 'DEC--TAN',
         'CRVAL1': 3.581704851882,
         'CRVAL2': -30.39197867265,
         'LONPOLE': 180.0,
         'LATPOLE': -30.39197867265,
         'MJDREF': 0.0,
         'RADESYS': 'ICRS'})

In [ ]:
_ = wcs_utils.draw_compass_mpl(a, wcs=w)

Now, let's try that in the Compass plugin. We will display this same image in Imviz. Open the Compass plugin to see the result.

In [ ]:
hdu = fits.ImageHDU(a, name='SCI')
hdu.header.update(w.to_header())

In [ ]:
imviz = Imviz()
imviz.load_data(hdu, data_label='jb5g05ubq_flt')
imviz.show()

## Case 4: GWCS

Again, we use random data. The GWCS is from https://gwcs.readthedocs.io/en/latest/#getting-started example.

In [ ]:
a = np.random.random((1024, 2048))

shift_by_crpix = models.Shift(-(2048 - 1) * u.pix) & models.Shift(-(1024 - 1) * u.pix)
matrix = np.array([[1.290551569736E-05, 5.9525007864732E-06],
                   [5.0226382102765E-06, -1.2644844123757E-05]])
rotation = models.AffineTransformation2D(matrix * u.deg, translation=[0, 0] * u.deg)
rotation.input_units_equivalencies = {"x": u.pixel_scale(1 * (u.deg / u.pix)),
                                      "y": u.pixel_scale(1 * (u.deg / u.pix))}
rotation.inverse = models.AffineTransformation2D(np.linalg.inv(matrix) * u.pix,
                                                 translation=[0, 0] * u.pix)
rotation.inverse.input_units_equivalencies = {"x": u.pixel_scale(1 * (u.pix / u.deg)),
                                              "y": u.pixel_scale(1 * (u.pix / u.deg))}
tan = models.Pix2Sky_TAN()
celestial_rotation = models.RotateNative2Celestial(
    5.63056810618 * u.deg, -72.05457184279 * u.deg, 180 * u.deg)
det2sky = shift_by_crpix | rotation | tan | celestial_rotation
det2sky.name = "linear_transform"
detector_frame = cf.Frame2D(name="detector", axes_names=("x", "y"), unit=(u.pix, u.pix))
sky_frame = cf.CelestialFrame(reference_frame=ICRS(), name='icrs', unit=(u.deg, u.deg))
pipeline = [(detector_frame, det2sky), (sky_frame, None)]
w = gwcs.wcs.WCS(pipeline)

In [ ]:
_ = wcs_utils.draw_compass_mpl(a, wcs=w)

Now, let's try that in the Compass plugin. We will display this same image in Imviz. Open the Compass plugin to see the result.

In [ ]:
data = Data(label='my_gwcs')
data.coords = w
component = Component.autotyped(a, units='MJy/sr')
_ = data.add_component(component=component, label='DATA')

In [ ]:
imviz = Imviz()
imviz.app.add_data(data, data.label)
imviz.app.add_data_to_viewer('imviz-0', data.label)
imviz.show()

## Case 5: Bad WCS

This image from the Imviz example notebook will give the following error when we attempt to calculate the compass in the backend:

    NoConvergence: 'WCS.all_world2pix' failed to converge to the requested accuracy.
    After 2 iterations, the solution is diverging at least for one input point.

Therefore, you will not see the N and E axes in the display.

In [ ]:
acs_47tuc_1 = download_file('https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:HST/product/jbqf03gjq_flc.fits', cache=True)

pf = fits.open(acs_47tuc_1)
a = pf[1].data
w = WCS(pf[1].header, pf)

In [ ]:
_ = wcs_utils.draw_compass_mpl(a, wcs=w)

Now, let's try that in the Compass plugin. We will display this same image in Imviz. Open the Compass plugin to see the result.

In [ ]:
imviz = Imviz()
imviz.load_data(acs_47tuc_1, data_label='acs_47tuc_1')
imviz.show()

## Case 6: No WCS

In this example, we will see how the Compass plugin behaves when image has no WCS. You will not see the N and E axes in the display.

In [ ]:
a = np.arange(100).reshape((10, 10))

In [ ]:
imviz = Imviz()
imviz.load_data(a, data_label='no_wcs')
imviz.show()

## Case 7: Dithered images

What about two images dithered by one pixel in X, and a third one with no WCS? We display them all. Open the Compass plugin and blink the images using `b` key.

In [ ]:
a = np.arange(100).reshape((10, 10))

# First data with WCS.
hdu1 = fits.ImageHDU(a, name='SCI')
hdu1.header.update({'CTYPE1': 'RA---TAN',
                    'CUNIT1': 'deg',
                    'CDELT1': -0.0002777777778,
                    'CRPIX1': 1,
                    'CRVAL1': 337.5202808,
                    'NAXIS1': 10,
                    'CTYPE2': 'DEC--TAN',
                    'CUNIT2': 'deg',
                    'CDELT2': 0.0002777777778,
                    'CRPIX2': 1,
                    'CRVAL2': -20.833333059999998,
                    'NAXIS2': 10})

# Second data with WCS, similar to above but dithered by 1 pixel in X.
hdu2 = fits.ImageHDU(a, name='SCI')
hdu2.header.update({'CTYPE1': 'RA---TAN',
                    'CUNIT1': 'deg',
                    'CDELT1': -0.0002777777778,
                    'CRPIX1': 2,
                    'CRVAL1': 337.5202808,
                    'NAXIS1': 10,
                    'CTYPE2': 'DEC--TAN',
                    'CUNIT2': 'deg',
                    'CDELT2': 0.0002777777778,
                    'CRPIX2': 1,
                    'CRVAL2': -20.833333059999998,
                    'NAXIS2': 10})

In [ ]:
imviz = Imviz()
imviz.load_data(hdu1, data_label='has_wcs_1')
imviz.load_data(hdu2, data_label='has_wcs_2')
imviz.load_data(a, data_label='no_wcs')
imviz.link_data(link_type='wcs')
imviz.show()

## Case 8: Galactic center image from astropy-data

In [ ]:
filename = download_file('https://github.com/astropy/astropy-data/raw/gh-pages/galactic_center/gc_2mass_k.fits', cache=True)

In [ ]:
imviz = Imviz()
imviz.load_data(filename, data_label='gc_2mass_k')
imviz.show()